# FINE TUNING OF OUR PREVIOUSLY TRAINED MODEL 


STEP 1: preprocess the dataset at disposal and import needed libraries 

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import tensorflow as tf
import numpy as np

SEED = 345
tf.random.set_seed(SEED)  

cwd = os.getcwd()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant', #da provare nearest
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

valid_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
dataset_dir = os.path.join(cwd, 'newDataset')

bs = 5

img_h=500
img_w=500

num_classes= 20

classes = [ 'owl',    # 0
            'galaxy', # 1
            'lightning', # 2
            'wine-bottle', # 3
            't-shirt', # 4
            'waterfall', # 5
            'sword', # 6
            'school-bus', # 7
            'calculator', # 8
            'sheet-music', # 9
            'airplanes', # 10
            'lightbulb', # 11
            'skyscraper', # 12
            'mountain-bike', # 13
            'fireworks', # 14
            'computer-monitor', # 15
            'bear', # 16
            'grand-piano', # 17
            'kangaroo', # 18
            'laptop']       # 19


# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               target_size=(img_h,img_w),
                                               batch_size=bs, 
                                               classes=classes,
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)  # targets are directly converted into one-hot vectors


class_indices = train_gen.class_indices
class_indices

## TRANSFER LEARNING 

we import the model trained before and fine tune it. The starting layer can be trimmed in order to better adapt the model 

In [ ]:
#define the input images dimesions and the directory of the model 
cwd = os.getcwd()
model_path = os.path.join(cwd, 'my_model')
Input _shape = (img_h, img_w, 3)

#load the entire model and check everything is correct 
from keras.models import load_model

new_model_ft= load_model(model_path)

new_model_ft.summary()
new_model_ft.summary.layers

In [ ]:
finetuning = True

if finetuning:
    freeze_until = 10 # layer from which we want to fine-tune
    
    for layer in new_model_ft.layers[:freeze_until]:
        layer.trainable = False
else:
    new_model_ft.trainable = False
    
model = tf.keras.Sequential()
model.add(new_model_ft)
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# I can visualise the model 
model.summary()

model.weights 

## OPTIMIZER, LEARNING RATE AND COMPILE 

In [ ]:
#define loss funciton, learning rate and optimizer 
loss = tf.keras.losses.CategoricalCrossentropy()

lr = 1e-2
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)

# Validation metrics
metrics = ['accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## CALLBACKS 

early stopping can be used or not, reduces risk of overfitting 

In [ ]:
import os
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join(cwd, 'classification_experiments')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN+TF'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'),
                                                   monitor='val_acc',
                                                   save_weights_only=False,
                                                   save_best_only=True,
                                                   period=5) 
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)

model.fit_generator(
    train_gen,
    validation_data = valid_gen, 
    epochs = 100)



## EXPORT IN CSV AND RESULTS 

test the model afterwards

In [ ]:
from datetime import datetime

def create_csv(results, results_dir='./output/'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [ ]:
import os
from PIL import Image
import numpy as np

image_filenames = next(os.walk('./startDataset/test'))[2]                          
results = {}

for image_name in image_filenames:
    img = Image.open('./Kaggle/test/'+image_name).convert('RGB')
    img = img.resize((img_w,img_h))
    img_array = np.array(img)
    img_array = img_array*1./255 #normalization
    img_array = np.expand_dims(img_array,0) #needed for fixed dim of input in the model
    output = model.predict(img_array)
    prediction = np.where(output == np.max(output))[1]  
    print('Image: '+str(image_name)+ ' Class: '+ list(class_indices.keys())[int(prediction)])
    results[image_name] = int(prediction)
    
create_csv(results)
